# Log into an Existing FABRIC node

This notebook will show you how to get an existing slice, get a node from the slice, get the IP and make the ssh command that you can use to access the node.

## Existing Slice

The rest of this notebook assumes you have an existing slice named "MySlice" that includes a node named "Node1". Set the varialbe betlow to match your slice.

In [ ]:
slice_name="MySlice"
node_name="Node1"

## Configure the Environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

In [ ]:
ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

## Create slice manager object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Get the Slice ID

In [ ]:
status, slices = slice_manager.slices(state="All")

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

Use a lambda filter to find all slices with the slice name. If there are mulitple slices with the same name this will assume the first one is the one want.

In [ ]:
slice = list(filter(lambda slices: slices.slice_name == slice_name, slices))[0]

slice_id = slice.slice_id

print("Slice {}".format(slice))
print("Slice ID {}".format(slice_id))

# Get the node's management IP

In [ ]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slivers {}".format(slivers))
    
management_ip = slivers[0].management_ip
print("Management IP: {}".format(management_ip))

## Create the ssh command

We have the management_ip. Now we print the SSH command that we can use to access the machine. We need to use the key path variable that we've defined at the time of slice creation. We have the path to our key stored in the "ssh_key_file_priv" variable.

The result of the command produced by the cell can be use a terminal to access the VM.

In [ ]:
print("ssh -i " + ssh_key_file_priv + " ubuntu@" + management_ip)